# 🧩 Main Live Data Advisory
Pipeline: Fetch live → Format → Run LLM → Display

In [ ]:
# Load components
%run live_data_loader.ipynb
%run LLM_loader.ipynb
%run format_prompt_from_live_data.ipynb

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from Model/mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:            

In [ ]:
# Example location: Karachi
# You can change the latitude and longitude to any location.
# To get data for multiple locations, create a list and loop over it using this same function.
lat, lon = 24.75, 67.02
weather_data = fetch_weather(lat, lon)
prompt = generate_prompt(weather_data)
print("Prompt:\n", prompt)

Prompt:
 
You are a weather advisory assistant working with NDMA.

Given the current real-time weather data below:

📍 Location: Karachi (24.75°N, 67.02°E)
🌡 Temperature: 29.51 °C
🥵 Feels Like: 35.95 °C
💧 Humidity: 79 %
🌬 Wind Speed: 22.86 km/h
📉 Pressure: 1003 hPa
🌫 Weather: Clear sky

Your job is to:
- Only generate risk warnings if values exceed safe thresholds
- Avoid including generic warnings if no severe risk exists
- Be concise and data-driven
- Give all in bullets

Output a clear advisory including:
1. Local weather summary
2. Only relevant risk warnings (if any)
3. Recommended actions

Response:



In [ ]:
# Generate advisory
# It is advised to save the chat history for future reference, especially to regenerate consistent or presized advisories.
def clean_advisory_output(text):
    # Strip unwanted characters or tokens
    cleaned = text.strip()
    for token in ["\\end{code}", "\\begin{code}", "```", "📋 Advisory:", "</code>"]:
        cleaned = cleaned.replace(token, "")
    return cleaned.strip()

if 'No data available' not in prompt:
    output = llm_model(
    prompt,
    max_tokens=300,
    temperature=0.7,
    stop=["</s>", "User:", "###"]
    )
    advisory_raw = output['choices'][0]['text']
    advisory_clean = clean_advisory_output(advisory_raw)
    print("\n✅ Final Advisory:\n", advisory_clean)

llama_perf_context_print:        load time =   10880.50 ms
llama_perf_context_print: prompt eval time =   10880.24 ms /   230 tokens (   47.31 ms per token,    21.14 tokens per second)
llama_perf_context_print:        eval time =   33058.09 ms /   284 runs   (  116.40 ms per token,     8.59 tokens per second)
llama_perf_context_print:       total time =   44119.46 ms /   514 tokens



✅ Final Advisory:
 **Local Weather Summary:**
Current weather in Karachi is clear with a temperature of 29.51 °C, feels like 35.95 °C, humidity of 79 %, wind speed of 22.86 km/h, and atmospheric pressure of 1003 hPa.

**Risk Warnings:**
- No severe heat wave or extreme temperature warning is triggered as the feels like temperature (35.95 °C) is within the safe range for human comfort.
- No heavy rainfall or flooding warning is issued as there is no significant precipitation forecasted.
- No severe wind warning is raised as wind speed (22.86 km/h) is below the warning threshold.

**Recommended Actions:**
1. Stay hydrated by drinking plenty of fluids, especially water.
2. Wear loose, light-colored and breathable clothing.
3. Limit outdoor activities during peak heat hours (12-3 PM).
4. Check on elderly family members, friends, and neighbors.
5. Keep indoor environment cool by using air conditioning, fans or natural ventilation.
6. Avoid direct sunlight as much as possible.
7. Avoid stre